In [3]:
import numpy as np
import pandas as pd
import redis

In [4]:
r = redis.StrictRedis(host="redis-13914.c322.us-east-1-2.ec2.cloud.redislabs.com",
                port=13914,
                password='h9fng3OkGwuY1FxPHguTBZ8rVt6uCdrx'
               )

In [5]:
r.ping()

True

In [6]:
r.keys()

[b'academy:register', b'KEY', b'MESSAGE', b'foo', b'mylist1', b'mylist']

### Load the data(numpt zip)

In [7]:
np_file = np.load('dataframe_students_teacher.npz',allow_pickle=True)
x_values = np_file['arr_0']
col_names = np_file['arr_1']

df = pd.DataFrame(x_values,columns=col_names)

In [8]:
df

,Name,Role,Facial Features
0,Angelina Jolie,Student,"[0.43242025, 1.0748498, -0.12862365, -1.833291..."
1,Barack Obama,Teacher,"[0.9485809, -0.5584051, 0.094820715, -1.279659..."
2,Chris Evans,Student,"[0.7201714, -1.4684148, -0.7781253, -1.4601073..."
3,Morgan Freeman,Teacher,"[0.48535094, -0.6192554, -0.4069932, 1.0687788..."
4,Scarlett Johansson,Student,"[0.19131956, -0.48843804, -1.690233, 0.4423967..."


In [9]:
df['name_role'] = df['Name']+'@'+df['Role']
df

,Name,Role,Facial Features,name_role
0,Angelina Jolie,Student,"[0.43242025, 1.0748498, -0.12862365, -1.833291...",Angelina Jolie@Student
1,Barack Obama,Teacher,"[0.9485809, -0.5584051, 0.094820715, -1.279659...",Barack Obama@Teacher
2,Chris Evans,Student,"[0.7201714, -1.4684148, -0.7781253, -1.4601073...",Chris Evans@Student
3,Morgan Freeman,Teacher,"[0.48535094, -0.6192554, -0.4069932, 1.0687788...",Morgan Freeman@Teacher
4,Scarlett Johansson,Student,"[0.19131956, -0.48843804, -1.690233, 0.4423967...",Scarlett Johansson@Student


In [10]:
records = df[['name_role','Facial Features']].to_dict(orient='records')
len(records)

5

### Save data in Redis
- redis hashes
- keyname = 'academy.register'

In [11]:
for record in records:
    name_role = record['name_role']
    vector = record['Facial Features']
    
    ## convert numpy Array
    vector_bytes = vector.tobytes()
    r.hset(name='academy:register',key=name_role,value=vector_bytes)

In [12]:
r.keys()

[b'academy:register', b'KEY', b'MESSAGE', b'foo', b'mylist1', b'mylist']

### Retrive Records from Redis

In [13]:
retrive_dict = r.hgetall(name='academy:register')
retrive_series = pd.Series(retrive_dict)
retrive_series

b'Angelina Jolie@Student'        b'0f\xdd>\xae\x94\x89?\xeb\xb5\x03\xbeN\xa9\xe...
b'Pratik Raut@Student'           b'1\x89\x9f\xbf\x03{\xef\xbe\xa4\xb4\xcf>\xa9^...
b'Barack Obama@Teacher'          b'3\xd6r?\xa3\xf3\x0e\xbf]1\xc2=\xde\xcb\xa3\x...
b'Chris Evans@Student'           b'\']8?\x04\xf5\xbb\xbf83G\xbf\xcc\xe4\xba\xbf...
b'Morgan Freeman@Teacher'        b'\xeb\x7f\xf8>\x86\x87\x1e\xbfja\xd0\xbe\xbe\...
b'Scarlett Johansson@Student'    b'F\xe9C>\x8d\x14\xfa\xbe\x8eY\xd8\xbf\xd3\x81...
dtype: object

In [14]:
#np.framebuffer(retriving_series[b'Morgan Freeman@Teacger'],dtype.float32).shape
retrive_series = retrive_series.apply(lambda x: np.frombuffer(x,dtype=np.float32))
index = retrive_series.index
index = list(map(lambda x: x.decode(),index))

In [15]:
retrive_series.index = index

In [16]:
retrive_df = retrive_series.to_frame().reset_index()
retrive_df.columns = ['name_role','facial_features']
retrive_df

,name_role,facial_features
0,Angelina Jolie@Student,"[0.43242025, 1.0748498, -0.12862365, -1.833291..."
1,Pratik Raut@Student,"[-1.2463742, -0.46773538, 0.40567505, 2.755777..."
2,Barack Obama@Teacher,"[0.9485809, -0.5584051, 0.094820715, -1.279659..."
3,Chris Evans@Student,"[0.7201714, -1.4684148, -0.7781253, -1.4601073..."
4,Morgan Freeman@Teacher,"[0.48535094, -0.6192554, -0.4069932, 1.0687788..."
5,Scarlett Johansson@Student,"[0.19131956, -0.48843804, -1.690233, 0.4423967..."


### Registration Form

In [17]:
import cv2
from insightface.app import FaceAnalysis

In [18]:
## configure face analysis
faceapp = FaceAnalysis(name='buffalo_sc',
                       root='insightface_model\models',
                       providers=['CPUExecutionProvider'])

faceapp.prepare(ctx_id=0,det_size=(640,640),det_thresh=0.5)
#warning dont set det_thresh < 0.3

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


### Step1: Collect Person and Role

In [19]:
#python input
person_name = input('Enter your name: ')
trails =3
for i in range(trails):
    role = input("""
                Please choose
                1.Student
                2.Teacher
                
                Enter the number either 1 or 2
                """)

    if role in ('1','2'):
        if role=='1':
            role = 'Student'
        else:
            role = 'Teacher'

        break

    else:
        print('Invalid entry try again')
        if i==3:
            print('Exceeds maximum trails')

key = person_name+'@'+role
print('Your name = ',person_name)
print('Your role = ',role)
print('KEy = ',key)


In [ ]:
person_name  ,role

NameError: name 'person_name' is not defined

### Collect Samples

In [ ]:
cap =  cv2.VideoCapture(0) ## 0 default camera 1 external camera
face_embeddings = []
sample = 0

while True:
    ret , frame = cap.read()
    if ret == False:
        print('Unable to read camera')
        break

    #get results from insightface model
    results = faceapp.get(frame,max_num=1)
    for res in results:
        sample += 1
        x1,y1,x2,y2 = res['bbox'].astype(int)
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),1)

        #facial features
        embeddings = res['embedding']
        face_embeddings.append(embeddings)

    if sample>=200:
        break

    cv2.imshow('frame',frame)
    if cv2.waitKey(1) == ord('q'): #this will trigger only when I press letter q (lowercase q) in my keyboard
        break

cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [ ]:
len(face_embeddings)

200

### Optimize Data & Save Data in Redis Database

In [ ]:
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [ ]:
#convert into bytes
x_mean_bytes = x_mean.tobytes()

In [ ]:
#save key and facial bytes into redis database
r.hset(name='academy:register',key=key,value=x_mean_bytes)

1